In [ ]:
# ============================================================================
# WavLM Ultimate 5-Fold CV
# 架构: Raw Audio -> WavLM (Partial Freeze) -> Weighted Sum -> Bi-LSTM -> Attn -> Focal Loss
# 新增特性:
# 1. 5-Fold Cross-Validation (Speaker Independent)
# 2. Weighted Focal Loss (手动增加 HAP 权重)
# ============================================================================


import os
# 离线模式配置
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["TRANSFORMERS_OFFLINE"] = "1" 

import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from transformers import Wav2Vec2FeatureExtractor, WavLMModel
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import gc
from tqdm.auto import tqdm  

# 检查 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# =====================
# 1. 配置参数
# =====================
MODEL_NAME_OR_PATH = "../wavlm_base_plus_local"
AUDIO_DIR = Path("../AudioWAV")
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_DIR = f"Output/WavLM_5Fold_{timestamp}"
os.makedirs(OUTPUT_DIR, exist_ok=True)

CONFIG = {
    "sr": 16000,
    "max_duration": 3.0,
    "batch_size": 128,     
    "num_epochs": 80,     
    "lr": 1e-4,           
    "backbone_lr": 1e-5,  
    "freeze_layers": 8,   
    "lstm_hidden": 256,   
    "n_folds": 5,         
    "hap_weight": 1.2     
}

# =====================
# 2. 数据准备
# =====================
def build_metadata(audio_dir):
    records = []
    if not audio_dir.exists():
        raise ValueError(f"Directory {audio_dir} does not exist!")
    for wav in audio_dir.glob("*.wav"):
        parts = wav.stem.split("_")
        if len(parts) == 4:
            records.append({"path": str(wav), "spk": parts[0], "emo": parts[2]})
    return pd.DataFrame(records)

print("Building metadata...")
meta = build_metadata(AUDIO_DIR)
le = LabelEncoder()
meta['label'] = le.fit_transform(meta['emo'])
CLASSES = le.classes_
NUM_CLASSES = len(CLASSES)
print(f"Total Samples: {len(meta)}")

hap_index = np.where(CLASSES == 'HAP')[0][0]

# =====================
# 3. Dataset
# =====================
class CREMADataset(Dataset):
    def __init__(self, df, processor, sr=16000, duration=3.0, augment=False):
        self.df = df; self.processor = processor; self.sr = sr
        self.target_len = int(sr * duration); self.augment = augment

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        path = self.df.iloc[idx]['path']
        wav, _ = librosa.load(path, sr=self.sr)
        if len(wav) > self.target_len:
            start = np.random.randint(0, len(wav) - self.target_len) if self.augment else (len(wav) - self.target_len) // 2
            wav = wav[start : start + self.target_len]
        else:
            wav = np.pad(wav, (0, self.target_len - len(wav)), mode='constant')
        if self.augment and np.random.random() < 0.3:
            wav += 0.005 * np.random.randn(len(wav))
        inputs = self.processor(wav, sampling_rate=self.sr, return_tensors="pt")
        return {"input_values": inputs.input_values.squeeze(0), "labels": torch.tensor(self.df.iloc[idx]['label'], dtype=torch.long)}

def collate_fn(batch):
    return {
        "input_values": torch.stack([item['input_values'] for item in batch]),
        "labels": torch.stack([item['labels'] for item in batch])
    }

try:
    processor = Wav2Vec2FeatureExtractor.from_pretrained(MODEL_NAME_OR_PATH, local_files_only=True)
except:
    processor = Wav2Vec2FeatureExtractor.from_pretrained("microsoft/wavlm-base-plus")

# =====================
# 4. 模型架构
# =====================
class SelfAttentionPooling(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.W = nn.Linear(input_dim, 1)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, batch_rep):
        att_logits = self.W(batch_rep)
        att_weights = self.softmax(att_logits)
        utt_rep = torch.sum(batch_rep * att_weights, dim=1)
        return utt_rep

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super().__init__()
        self.gamma = gamma; self.alpha = alpha; self.reduction = reduction
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss
        if self.reduction == 'mean': return focal_loss.mean()
        else: return focal_loss.sum()

class WavLMUltimateModel(nn.Module):
    def __init__(self, model_name, num_classes, freeze_layers=8, lstm_hidden=256):
        super().__init__()
        try:
            self.wavlm = WavLMModel.from_pretrained(model_name, local_files_only=True, output_hidden_states=True)
        except:
            self.wavlm = WavLMModel.from_pretrained("microsoft/wavlm-base-plus", output_hidden_states=True)
        self.wavlm.feature_extractor._freeze_parameters()
        for layer in self.wavlm.encoder.layers[:freeze_layers]:
            for param in layer.parameters(): param.requires_grad = False
        self.layer_weights = nn.Parameter(torch.ones(13) / 13) 
        self.lstm = nn.LSTM(768, lstm_hidden, num_layers=2, batch_first=True, bidirectional=True, dropout=0.2)
        self.attn_pooling = SelfAttentionPooling(lstm_hidden * 2)
        self.classifier = nn.Sequential(
            nn.Linear(lstm_hidden * 2, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(0.3), nn.Linear(256, num_classes)
        )
    def forward(self, input_values):
        outputs = self.wavlm(input_values)
        all_layers = torch.stack(outputs.hidden_states, dim=-1) 
        norm_weights = F.softmax(self.layer_weights, dim=0)
        weighted_feature = (all_layers * norm_weights).sum(dim=-1)
        lstm_out, _ = self.lstm(weighted_feature)
        pooled_feature = self.attn_pooling(lstm_out)
        logits = self.classifier(pooled_feature)
        return logits

# =====================
# 5. 5-Fold 训练主循环 
# =====================
def train_one_fold(fold_idx, train_df, val_df):
    print(f"\n{'='*20} Fold {fold_idx+1}/{CONFIG['n_folds']} {'='*20}")
    print(f"Train: {len(train_df)} | Val: {len(val_df)}")
    
    # DataLoader
    train_ds = CREMADataset(train_df, processor, augment=True)
    val_ds = CREMADataset(val_df, processor, augment=False)
    train_loader = DataLoader(train_ds, batch_size=CONFIG['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=0)  # 4 -> 0
    val_loader = DataLoader(val_ds, batch_size=CONFIG['batch_size'], shuffle=False, collate_fn=collate_fn, num_workers=0)     # 4 -> 0
    
    # Model
    model = WavLMUltimateModel(MODEL_NAME_OR_PATH, NUM_CLASSES, freeze_layers=CONFIG['freeze_layers'], lstm_hidden=CONFIG['lstm_hidden'])
    model.to(device)
    
    # Loss & Optimizer
    weights = torch.ones(NUM_CLASSES).to(device)
    weights[hap_index] = CONFIG['hap_weight']
    criterion = FocalLoss(gamma=2.0, alpha=weights)
    optimizer = torch.optim.AdamW([
        {'params': model.wavlm.parameters(), 'lr': CONFIG['backbone_lr']},
        {'params': model.layer_weights, 'lr': 1e-3},
        {'params': model.lstm.parameters(), 'lr': CONFIG['lr']},
        {'params': model.attn_pooling.parameters(), 'lr': CONFIG['lr']},
        {'params': model.classifier.parameters(), 'lr': CONFIG['lr']}
    ], weight_decay=1e-3)
    
    # Loop
    best_f1 = 0.0
    best_acc = 0.0
    fold_save_path = os.path.join(OUTPUT_DIR, f"fold_{fold_idx+1}_best.pth")
    
    for epoch in range(CONFIG['num_epochs']):
        model.train()
        train_loss = 0
        
        # leave=False: 跑完一个Epoch后进度条消失
        pbar = tqdm(train_loader, desc=f"Fold {fold_idx+1} Ep {epoch+1}/{CONFIG['num_epochs']}", leave=False)
        
        for batch in pbar:
            inputs = batch['input_values'].to(device)
            labels = batch['labels'].to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            current_loss = loss.item()
            train_loss += current_loss
            
            # 实时更新进度条上的 Loss 显示
            pbar.set_postfix({"loss": f"{current_loss:.4f}"})
            
        # Validation
        model.eval()
        val_preds, val_labels = [], []
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch['input_values'].to(device)
                labels = batch['labels'].to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                val_preds.extend(predicted.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_f1 = f1_score(val_labels, val_preds, average='macro')
        val_acc = accuracy_score(val_labels, val_preds)
        
        # 打印简要日志
        print(f"  Ep {epoch+1}: Loss {train_loss/len(train_loader):.4f} | Val F1 {val_f1:.4f} | Val Acc {val_acc:.4f}", end="")
        
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_acc = val_acc
            torch.save(model.state_dict(), fold_save_path)
            print(" -> Saved ★")
        else:
            print("")
    
    print(f"Fold {fold_idx+1} Best F1: {best_f1:.4f}")
    
    del model, optimizer, train_loader, val_loader
    torch.cuda.empty_cache()
    gc.collect()
    
    return best_f1, best_acc, fold_save_path

# =====================
# 6. 主程序
# =====================
gkf = GroupKFold(n_splits=CONFIG['n_folds'])
fold_results, fold_paths = [], []

for fold_idx, (train_idx, val_idx) in enumerate(gkf.split(meta, meta['label'], groups=meta['spk'])):
    train_df = meta.iloc[train_idx].reset_index(drop=True)
    val_df = meta.iloc[val_idx].reset_index(drop=True)
    f1, acc, path = train_one_fold(fold_idx, train_df, val_df)
    fold_results.append({'fold': fold_idx+1, 'f1': f1, 'acc': acc})
    fold_paths.append(path)

# =====================
# 7. 汇总与评估
# =====================
print("\n" + "="*30)
print("5-FOLD CV RESULTS SUMMARY")
print("="*30)
f1_scores = [r['f1'] for r in fold_results]
print(f"F1 Scores: {[f'{s:.4f}' for s in f1_scores]}")
print(f"Average F1: {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")

# 全局混淆矩阵
print("\nGenerating Global Confusion Matrix...")
all_preds, all_targets = [], []

for fold_idx, (train_idx, val_idx) in enumerate(gkf.split(meta, meta['label'], groups=meta['spk'])):
    val_df = meta.iloc[val_idx].reset_index(drop=True)
    val_loader = DataLoader(CREMADataset(val_df, processor, augment=False), batch_size=CONFIG['batch_size'], shuffle=False, collate_fn=collate_fn)
    
    model = WavLMUltimateModel(MODEL_NAME_OR_PATH, NUM_CLASSES, freeze_layers=CONFIG['freeze_layers'], lstm_hidden=CONFIG['lstm_hidden'])
    model.load_state_dict(torch.load(fold_paths[fold_idx]))
    model.to(device)
    model.eval()
    
    with torch.no_grad():
        for batch in val_loader:
            inputs = batch['input_values'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())
    del model
    torch.cuda.empty_cache()

print(classification_report(all_targets, all_preds, target_names=CLASSES, digits=4))
cm = confusion_matrix(all_targets, all_preds)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CLASSES, yticklabels=CLASSES)
plt.title(f"5-Fold Global Confusion Matrix (Avg F1: {np.mean(f1_scores):.4f})")
plt.savefig(os.path.join(OUTPUT_DIR, "global_confusion_matrix.png"))
plt.show()
print(f"\nAll results saved to: {OUTPUT_DIR}")

2025-12-06 10:13:32.671827: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-06 10:13:32.774599: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-06 10:13:33.824763: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/root/miniconda3/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still ch

Using device: cuda
Building metadata...
Total Samples: 7442

==================== Fold 1/5 ====================
Train: 5966 | Val: 1476


/root/miniconda3/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Fold 1 Ep 1/80:   0%|          | 0/47 [00:00<?, ?it/s]

  Ep 1: Loss 0.9706 | Val F1 0.4195 | Val Acc 0.4539 -> Saved ★


...

Fold 5 Ep 78/80:   0%|          | 0/47 [00:00<?, ?it/s]

  Ep 78: Loss 0.0784 | Val F1 0.7307 | Val Acc 0.7299


Fold 5 Ep 79/80:   0%|          | 0/47 [00:00<?, ?it/s]

  Ep 79: Loss 0.0898 | Val F1 0.7453 | Val Acc 0.7456


Fold 5 Ep 80/80:   0%|          | 0/47 [00:00<?, ?it/s]

In [1]:
# ============================================================================
# WavLM Ultimate 5-Fold CV (Full Version)
# 包含功能:
# 1. 核心修复: Mask Downsampling & Augmentation Logic
# 2. 显存优化: Gradient Accumulation
# 3. 增强监控: 实时显示 LR, Recall, Precision, Accuracy
# 4. 可视化: 自动绘制 Loss/F1 曲线图 & 混淆矩阵
# ============================================================================

import os
# 设置离线模式和镜像
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["TRANSFORMERS_OFFLINE"] = "1" 

import warnings
# 忽略所有警告信息，还你一个清爽的控制台
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler

from pathlib import Path
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score, precision_recall_fscore_support

from transformers import Wav2Vec2FeatureExtractor, WavLMModel, get_linear_schedule_with_warmup
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import gc
import random
from tqdm.auto import tqdm

# 防止绘图报错
plt.switch_backend('agg') 

# =====================
# 0. 全局配置与种子
# =====================
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

SEED = 42
seed_everything(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_NAME_OR_PATH = "../wavlm_base_plus_local"
AUDIO_DIR = Path("../AudioWAV") # 请确保路径正确

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_DIR = f"Output/WavLM_Vis_{timestamp}"

CONFIG = {
    "sr": 16000,
    "max_duration": 3.0,
    "batch_size": 256,      
    "accum_steps": 2,      # 梯度累积
    "num_epochs": 80,      
    "lr": 1e-4,            
    "backbone_lr": 1e-5,   
    "freeze_layers": 8,    
    "lstm_hidden": 256,    
    "n_folds": 5,          
    "hap_weight": 1.2,     
    "warmup_ratio": 0.1
}

# =====================
# 1. 可视化工具函数
# =====================
def plot_history(history, fold_idx, output_dir):
    """绘制 Loss, Accuracy, F1, LR 曲线"""
    epochs = range(1, len(history['train_loss']) + 1)
    
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    
    # 1. Loss
    axes[0].plot(epochs, history['train_loss'], label='Train Loss', marker='o', markersize=3)
    axes[0].plot(epochs, history['val_loss'], label='Val Loss', marker='o', markersize=3)
    axes[0].set_title(f'Fold {fold_idx+1} Loss Curve')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 2. Metrics (F1 & Acc)
    axes[1].plot(epochs, history['val_f1'], label='Val F1', color='orange', marker='s', markersize=3)
    axes[1].plot(epochs, history['val_acc'], label='Val Acc', color='green', marker='^', markersize=3)
    axes[1].set_title(f'Fold {fold_idx+1} Performance')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Score')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    # 3. Learning Rate
    axes[2].plot(epochs, history['lr'], label='Learning Rate', color='purple', linestyle='--')
    axes[2].set_title(f'Learning Rate Schedule')
    axes[2].set_xlabel('Epoch')
    axes[2].set_yscale('log')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"fold_{fold_idx+1}_history.png"), dpi=300)
    plt.close()

def plot_confusion_matrix(y_true, y_pred, classes, fold_idx, output_dir):
    """绘制混淆矩阵"""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=classes, yticklabels=classes)
    plt.title(f'Fold {fold_idx+1} Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"fold_{fold_idx+1}_cm.png"), dpi=300)
    plt.close()

# =====================
# 2. 数据准备
# =====================
def build_metadata(audio_dir):
    records = []
    if not audio_dir.exists():
        raise ValueError(f"Directory {audio_dir} does not exist!")
    for wav in audio_dir.glob("*.wav"):
        parts = wav.stem.split("_")
        if len(parts) == 4:
            records.append({"path": str(wav), "spk": parts[0], "emo": parts[2]})
    return pd.DataFrame(records)

# =====================
# 3. Dataset (修复增强逻辑)
# =====================
class CREMADataset(Dataset):
    def __init__(self, df, processor, sr=16000, duration=3.0, augment=False):
        self.df = df
        self.processor = processor
        self.sr = sr
        self.target_len = int(sr * duration)
        self.augment = augment

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.iloc[idx]['path']
        # 读取音频
        wav, _ = librosa.load(path, sr=self.sr)
        
        # 1. 随机裁剪 (如果音频过长)
        if len(wav) > self.target_len:
            if self.augment:
                start = np.random.randint(0, len(wav) - self.target_len)
            else:
                start = (len(wav) - self.target_len) // 2
            wav = wav[start : start + self.target_len]
        
        # 2. 增强 (仅针对有效语音部分，且未 Padding 前)
        if self.augment:
            # 简单加噪
            if np.random.random() < 0.3:
                noise = np.random.randn(len(wav))
                wav += 0.005 * noise
            # 可以添加 TimeStretch 或 PitchShift 等
            
        # 3. Padding (补 0)
        if len(wav) < self.target_len:
            wav = np.pad(wav, (0, self.target_len - len(wav)), mode='constant')

        # 4. Processor
        inputs = self.processor(wav, sampling_rate=self.sr, return_tensors="pt")
        
        return {
            "input_values": inputs.input_values.squeeze(0), 
            "attention_mask": inputs.attention_mask.squeeze(0), 
            "labels": torch.tensor(self.df.iloc[idx]['label'], dtype=torch.long)
        }

def collate_fn(batch):
    return {
        "input_values": torch.stack([item['input_values'] for item in batch]),
        "attention_mask": torch.stack([item['attention_mask'] for item in batch]),
        "labels": torch.stack([item['labels'] for item in batch])
    }

# =====================
# 4. 模型架构 (修复 Mask 下采样)
# =====================
class SelfAttentionPooling(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.W = nn.Linear(input_dim, 1)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, batch_rep, attention_mask=None):
        """
        batch_rep: (B, T, D)
        attention_mask: (B, T)
        """
        att_logits = self.W(batch_rep).squeeze(-1) # (B, T)
        
        if attention_mask is not None:
            # 将 Padding 部分的 Attention Score 设为极小值
            att_logits = att_logits.masked_fill(attention_mask == 0, -1e4)
            
        att_weights = self.softmax(att_logits).unsqueeze(-1)
        utt_rep = torch.sum(batch_rep * att_weights, dim=1)
        return utt_rep

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()

class WavLMUltimateModel(nn.Module):
    def __init__(self, model_name, num_classes, freeze_layers=8, lstm_hidden=256):
        super().__init__()
        try:
            self.wavlm = WavLMModel.from_pretrained(model_name, local_files_only=True, output_hidden_states=True)
        except:
            self.wavlm = WavLMModel.from_pretrained("microsoft/wavlm-base-plus", output_hidden_states=True)
            
        self.wavlm.feature_extractor._freeze_parameters()
        
        # 冻结部分 Transformer 层
        if freeze_layers > 0:
            for layer in self.wavlm.encoder.layers[:freeze_layers]:
                for param in layer.parameters():
                    param.requires_grad = False
        
        self.layer_weights = nn.Parameter(torch.ones(13) / 13) 
        self.lstm = nn.LSTM(768, lstm_hidden, num_layers=2, batch_first=True, bidirectional=True, dropout=0.2)
        self.attn_pooling = SelfAttentionPooling(lstm_hidden * 2)
        self.classifier = nn.Sequential(
            nn.Linear(lstm_hidden * 2, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def _get_downsampled_mask(self, attention_mask):
        """利用 WavLM 内部逻辑计算经过 CNN 后的 Mask 长度"""
        # 计算有效长度
        input_lengths = attention_mask.sum(dim=1).long()
        output_lengths = self.wavlm._get_feat_extract_output_lengths(input_lengths)
        
        batch_size = attention_mask.shape[0]
        # 获取当前特征对应的最大长度 (通常是 input_len / 320)
        max_len = self.wavlm._get_feat_extract_output_lengths(torch.tensor(attention_mask.shape[1])).item()
        
        new_mask = torch.zeros((batch_size, max_len), device=attention_mask.device)
        for i in range(batch_size):
            new_mask[i, :output_lengths[i]] = 1
        return new_mask

    def forward(self, input_values, attention_mask=None):
        # 1. Backbone
        outputs = self.wavlm(input_values, attention_mask=attention_mask)
        
        # 2. Weighted Sum of Layers
        hidden_states = torch.stack(outputs.hidden_states, dim=-1) # (B, T, D, Layers)
        norm_weights = F.softmax(self.layer_weights, dim=0)
        weighted_feature = (hidden_states * norm_weights).sum(dim=-1) # (B, T, D)
        
        # 3. LSTM
        lstm_out, _ = self.lstm(weighted_feature)
        
        # 4. 计算下采样后的 Mask 并进行 Pooling
        if attention_mask is not None:
            downsampled_mask = self._get_downsampled_mask(attention_mask)
            # 确保 mask 长度和 lstm_out 长度一致 (防止由于取整导致的细微差异)
            if downsampled_mask.shape[1] > lstm_out.shape[1]:
                downsampled_mask = downsampled_mask[:, :lstm_out.shape[1]]
            elif downsampled_mask.shape[1] < lstm_out.shape[1]:
                # 极少情况，padding补齐
                diff = lstm_out.shape[1] - downsampled_mask.shape[1]
                downsampled_mask = F.pad(downsampled_mask, (0, diff))
        else:
            downsampled_mask = None
            
        pooled_feature = self.attn_pooling(lstm_out, downsampled_mask)
        
        # 5. Classifier
        logits = self.classifier(pooled_feature)
        return logits

# =====================
# 5. 训练函数 (含详细日志与可视化)
# =====================
def train_one_fold(fold_idx, train_df, val_df, meta_classes, hap_index):
    # 初始化 Processor
    try: processor = Wav2Vec2FeatureExtractor.from_pretrained(MODEL_NAME_OR_PATH, local_files_only=True)
    except: processor = Wav2Vec2FeatureExtractor.from_pretrained("microsoft/wavlm-base-plus")

    # Dataset & Loader
    train_ds = CREMADataset(train_df, processor, augment=True)
    val_ds = CREMADataset(val_df, processor, augment=False)
    
    train_loader = DataLoader(train_ds, batch_size=CONFIG['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=0)
    val_loader = DataLoader(val_ds, batch_size=CONFIG['batch_size'], shuffle=False, collate_fn=collate_fn, num_workers=0)
    
    # Model
    model = WavLMUltimateModel(MODEL_NAME_OR_PATH, len(meta_classes), 
                              freeze_layers=CONFIG['freeze_layers'], 
                              lstm_hidden=CONFIG['lstm_hidden'])
    model.to(device)
    
    # Loss & Optimizer
    weights = torch.ones(len(meta_classes)).to(device)
    weights[hap_index] = CONFIG['hap_weight']
    criterion = FocalLoss(gamma=2.0, alpha=weights)
    
    optimizer = torch.optim.AdamW([
        {'params': model.wavlm.parameters(), 'lr': CONFIG['backbone_lr']},
        {'params': model.layer_weights, 'lr': 1e-3},
        {'params': model.lstm.parameters(), 'lr': CONFIG['lr']},
        {'params': model.attn_pooling.parameters(), 'lr': CONFIG['lr']},
        {'params': model.classifier.parameters(), 'lr': CONFIG['lr']}
    ], weight_decay=1e-3)
    
    total_steps = len(train_loader) * CONFIG['num_epochs'] // CONFIG['accum_steps']
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(total_steps*CONFIG['warmup_ratio']), num_training_steps=total_steps)
    scaler = GradScaler()
    
    # Logging vars
    best_f1 = 0.0
    fold_save_path = os.path.join(OUTPUT_DIR, f"fold_{fold_idx+1}_best.pth")
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': [], 'val_f1': [], 'val_recall': [], 'lr': []}
    
    print(f"\n{'='*10} Fold {fold_idx+1} Started {'='*10}")
    
    for epoch in range(CONFIG['num_epochs']):
        # --- TRAIN ---
        model.train()
        train_loss, train_correct, train_total = 0, 0, 0
        
        optimizer.zero_grad()
        pbar = tqdm(train_loader, desc=f"Fold {fold_idx+1} Ep {epoch+1}", leave=False)
        
        for step, batch in enumerate(pbar):
            inputs = batch['input_values'].to(device)
            masks = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            with autocast():
                outputs = model(inputs, attention_mask=masks)
                loss = criterion(outputs, labels)
                loss = loss / CONFIG['accum_steps']
            
            scaler.scale(loss).backward()
            
            if (step + 1) % CONFIG['accum_steps'] == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad()
            
            # Record
            curr_loss = loss.item() * CONFIG['accum_steps']
            train_loss += curr_loss
            _, predicted = torch.max(outputs.detach(), 1)
            train_correct += (predicted == labels).sum().item()
            train_total += labels.size(0)
            
            pbar.set_postfix({"loss": f"{curr_loss:.4f}"})
            
        avg_train_loss = train_loss / len(train_loader)
        avg_train_acc = train_correct / train_total
        current_lr = optimizer.param_groups[0]['lr']
        
        # --- VALIDATION ---
        model.eval()
        val_loss = 0
        val_preds, val_labels = [], []
        
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch['input_values'].to(device)
                masks = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                with autocast():
                    outputs = model(inputs, attention_mask=masks)
                    v_loss = criterion(outputs, labels)
                
                val_loss += v_loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_preds.extend(predicted.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        avg_val_loss = val_loss / len(val_loader)
        
        # 计算详细指标
        precision, recall, f1, _ = precision_recall_fscore_support(val_labels, val_preds, average='macro', zero_division=0)
        acc = accuracy_score(val_labels, val_preds)
        
        # --- 保存历史 & 打印 ---
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['train_acc'].append(avg_train_acc)
        history['val_acc'].append(acc)
        history['val_f1'].append(f1)
        history['val_recall'].append(recall)
        history['lr'].append(current_lr)
        
        print(f" Ep {epoch+1:02d} | "
              f"T_Loss: {avg_train_loss:.4f} Acc: {avg_train_acc:.4f} | "
              f"V_Loss: {avg_val_loss:.4f} Acc: {acc:.4f} F1: {f1:.4f} Rec: {recall:.4f} | "
              f"LR: {current_lr:.2e}", end="")
        
        if f1 > best_f1:
            best_f1 = f1
            best_acc = acc
            torch.save(model.state_dict(), fold_save_path)
            print(" -> ★ Saved")
        else:
            print("")
            
    # --- END OF FOLD EVALUATION ---
    print(f"\nGenerating detailed report for Fold {fold_idx+1}...")
    model.load_state_dict(torch.load(fold_save_path))
    model.eval()
    
    final_preds, final_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            inputs = batch['input_values'].to(device)
            masks = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            with autocast():
                outputs = model(inputs, attention_mask=masks)
            _, predicted = torch.max(outputs.data, 1)
            final_preds.extend(predicted.cpu().numpy())
            final_labels.extend(labels.cpu().numpy())
    
    # 1. Classification Report
    print(classification_report(final_labels, final_preds, target_names=meta_classes, digits=4))
    
    # 2. Visualize History
    plot_history(history, fold_idx, OUTPUT_DIR)
    
    # 3. Visualize Confusion Matrix
    plot_confusion_matrix(final_labels, final_preds, meta_classes, fold_idx, OUTPUT_DIR)
    
    # Cleanup
    del model, optimizer, scaler, scheduler
    torch.cuda.empty_cache()
    gc.collect()
    
    return best_f1, best_acc

# =====================
# 6. 主程序
# =====================
if __name__ == '__main__':
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    print("Building metadata...")
    try:
        meta = build_metadata(AUDIO_DIR)
        print(f"Dataset Loaded: {len(meta)} samples")
    except ValueError as e:
        print(e)
        exit()

    le = LabelEncoder()
    meta['label'] = le.fit_transform(meta['emo'])
    CLASSES = le.classes_
    NUM_CLASSES = len(CLASSES)
    hap_index = np.where(CLASSES == 'HAP')[0][0]
    
    gkf = GroupKFold(n_splits=CONFIG['n_folds'])
    fold_results = []
    
    for fold_idx, (train_idx, val_idx) in enumerate(gkf.split(meta, meta['label'], groups=meta['spk'])):
        train_df = meta.iloc[train_idx].reset_index(drop=True)
        val_df = meta.iloc[val_idx].reset_index(drop=True)
        
        f1, acc = train_one_fold(fold_idx, train_df, val_df, CLASSES, hap_index)
        fold_results.append({'f1': f1, 'acc': acc})

    print("\n" + "="*30)
    print(f"5-FOLD CV RESULTS SUMMARY (Logged in {OUTPUT_DIR})")
    print("="*30)
    for i, res in enumerate(fold_results):
        print(f"Fold {i+1}: F1 {res['f1']:.4f}, Acc {res['acc']:.4f}")
        
    avg_f1 = np.mean([r['f1'] for r in fold_results])
    avg_acc = np.mean([r['acc'] for r in fold_results])
    print(f"\nAverage F1: {avg_f1:.4f}")
    print(f"Average Acc: {avg_acc:.4f}")

2025-12-07 08:44:35.556023: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-07 08:44:35.642164: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-07 08:44:36.712997: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Building metadata...
Dataset Loaded: 7442 samples

========== Fold 1 Started ==========


Fold 1 Ep 1:   0%|          | 0/24 [00:00<?, ?it/s]

 Ep 01 | T_Loss: 1.4177 Acc: 0.1857 | V_Loss: 1.3198 Acc: 0.1707 F1: 0.0486 Rec: 0.1667 | LR: 1.25e-06 -> ★ Saved


Fold 1 Ep 2:   0%|          | 0/24 [00:00<?, ?it/s]

...

 Ep 78 | T_Loss: 0.2075 Acc: 0.8403 | V_Loss: 0.5351 Acc: 0.6912 F1: 0.6837 Rec: 0.6970 | LR: 2.78e-07


Fold 5 Ep 79:   0%|          | 0/24 [00:00<?, ?it/s]

 Ep 79 | T_Loss: 0.2106 Acc: 0.8408 | V_Loss: 0.5245 Acc: 0.6912 F1: 0.6846 Rec: 0.6966 | LR: 1.39e-07


Fold 5 Ep 80:   0%|          | 0/24 [00:00<?, ?it/s]

 Ep 80 | T_Loss: 0.2021 Acc: 0.8493 | V_Loss: 0.5356 Acc: 0.6871 F1: 0.6795 Rec: 0.6928 | LR: 0.00e+00

Generating detailed report for Fold 5...
              precision    recall  f1-score   support

         ANG     0.6715    0.9283    0.7793       251
         DIS     0.7313    0.5857    0.6504       251
         FEA     0.7059    0.5737    0.6330       251
         HAP     0.9353    0.5179    0.6667       251
         NEU     0.7118    0.9535    0.8151       215
         SAD     0.6220    0.7211    0.6679       251

    accuracy                         0.7075      1470
   macro avg     0.7296    0.7134    0.7021      1470
weighted avg     0.7301    0.7075    0.6993      1470


5-FOLD CV RESULTS SUMMARY (Logged in Output/WavLM_Vis_20251207_084438)
Fold 1: F1 0.7289, Acc 0.7304
Fold 2: F1 0.6819, Acc 0.6854
Fold 3: F1 0.7782, Acc 0.7783
Fold 4: F1 0.7234, Acc 0.7270
Fold 5: F1 0.7021, Acc 0.7075

Average F1: 0.7229
Average Acc: 0.7257
